In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

page = requests.get('https://www.fasttrack.co.uk/league-tables/fast-track-100/league-table/')

html = BeautifulSoup(page.text, 'html.parser')

table = html.find('table', {'class': 'tableSorter2'})

tbody = table.find('tbody')

table_rows = tbody.find_all('tr')

data = []

for row in table_rows:
    row_data = []
    
    table_data = row.find_all('td')
    
    for i, td in enumerate(table_data):
        if i == 1:
            company_name = (td.find('a')).getText()
            company_description = (td.getText()).replace(company_name,'')

            company_info_page_link = td.find('a').get('href')
            company_info_page = requests.get(company_info_page_link)
            company_info_html = BeautifulSoup(company_info_page.text, 'html.parser')
            company_info_table = company_info_html.find('table')
            company_info_table_rows = company_info_table.find_all('tr')
            company_website_address = company_info_table_rows[-1].find('a').get('href')

            row_data.append(company_name)
            row_data.append(company_description)
            row_data.append(company_website_address)
        else:
            row_data.append(td.getText())
        
    data.append(row_data)
    print
    
df = pd.DataFrame(data)
df.columns = ['Rank', 'Company Name', 'Company Description', 'Website', 'Location', 'Year End', 'Sales Rise', 'Latest sales', 'Staff', 'Comment']
df.to_csv('results.csv', index=None, header=True)